In [17]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [18]:
load_dotenv(override=True)

True

In [19]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/chroma"
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [20]:
loader = PyPDFLoader(file)
file = loader.load()

# Recursive TextSplit

In [21]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [22]:
documents = recur_split.split_documents(file)

# Embbiding (OpenAI)

In [23]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [24]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# LLM (OpenAI)

In [25]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Chain

In [26]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

# Question

In [27]:
question = "O que é Identificar oportunidades?"

# Answer

In [28]:
chat_chain.invoke({"query": question})

{'query': 'O que é Identificar oportunidades?',
 'result': 'Identificar oportunidades é usar a sua imaginação e habilidades para visualizar oportunidades de criação de valor, como a identificação de necessidades e desafios que precisam ser atendidos.'}

# Prompt

In [29]:
prompt = PromptTemplate.from_template(
"""
Você é um assistente educacional que retorna competências empreendedoras.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
**Disciplina**;
**Conteúdo**;
**Objetivo da Aula**.

Com base no documento (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Traga somente 3 competências empreendedoras que estão no documento abaixo e somente do documento (que está no contexto abaixo).
Retorne apenas os nomes das competências, entre a primeira e a segunda separados por vírgula, e entre a segunda e a terceira separada por 'e'. \n\n

Exemplo: \n
Pergunta: "**Disciplina** <Matemática>, sendo o **Conteúdo** <Frações> e o **Objetivo da Aula** é <Calcular juros>." \n
Resposta: Identificar oportunidades, Mobilização de recursos e Educação financeira e econômica. \n\n

Contexto: {context} \n\n
Pergunta: {question}
"""
)

# Chain

In [30]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [ ]:
# question = "**Disciplina** <Matemática>, sendo o **Conteúdo** <Frações> e o **Objetivo da Aula** é <Utilização para calcular taxa de juros>."

# Answer

In [ ]:
# chat_chain.invoke({"query": question})

{'query': '**Disciplina** <Matemática>, sendo o **Conteúdo** <Frações> e o **Objetivo da Aula** é <Utilização para calcular taxa de juros>.',
 'result': 'Educação financeira e econômica, Mobilização de pessoas e Visão.',
 'source_documents': [Document(metadata={'creator': 'Microsoft® Word para Microsoft 365', 'source': '../pdf/entrecomp.pdf', 'page': 1, 'total_pages': 2, 'page_label': '2', 'moddate': '2025-04-28T16:59:30-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'author': 'Roselaine Monteiro Moraes', 'creationdate': '2025-04-28T16:59:30-03:00'}, page_content='Educação financeira e econômica: é desenvolver conhecimentos financeiros \ne econômicos, saber calcular o custo de transformar uma ideia numa atividade criadora \nde valor. Planejar, colocar em prática e avaliar decisões financeiras ao longo do tempo. \nGerir as finanças para assegurar que a atividade de criação de valor possa manter-se a \nlongo prazo. \n \nMobilização de pessoas: é inspirar, entusiasmar e mobiliz